# MISTy Analysis for Spatial Transcriptomics and Metal Abundance Data

In [ ]:
### This Jupyter Notebook performs a comprehensive MISTy (Multi-View Interactions in Spatial Transcriptomics) analysis. The analysis integrates single-cell (SC) and spatial transcriptomics (ST) data with metal abundance measurements to explore cellular interactions and pathway-level influences.

# Import necessary libraries
import scanpy as sc

In [ ]:
import scanpy as sc
import squidpy as sq
import decoupler as dc
import plotnine as p9
import liana as li
from liana.method import MistyData
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from mofapy2.run.entry_point import entry_point
import h5py
import mudata

## Load data files

In [ ]:
adata_st = sc.read_h5ad('ST_data_created_adata.h5ad')
adata_st.var_names_make_unique()

In [ ]:
sc_adata = pd.read_pickle("./SC_deconvolved_ST.pkl")

In [ ]:
data = pd.read_pickle("MISTy_model_all_data.pkl")

In [ ]:
spatial_tr = pd.read_pickle("MISTy_model_all_data_filtered.pkl")

In [ ]:
metals = pd.read_pickle("MISTy_model_metals_data.pkl")

## Run progeny pathway analysis

In [ ]:
dc.run_mlm(
    mat=adata_st,
    net=dc.get_progeny(organism='human', top=500),
    source='source',
    target='target',
    weight='weight',
    verbose=True,
    use_raw=False,
)

### Convert estimated pathways into an AnnData object

In [ ]:
acts_progeny = li.ut.obsm_to_adata(adata_st, 'mlm_estimate')

### Convert image colors

In [ ]:
acts_progeny.uns["spatial"]["092842"]["images"]['hires'] = cv2.cvtColor(
    acts_progeny.uns["spatial"]["092842"]["images"]['hires'], cv2.COLOR_BGR2RGB
)

## Spatial neighborhood analysis

In [ ]:
li.ut.spatial_neighbors(adata_st, cutoff=0.1, bandwidth=200, set_diag=False)
li.ut.spatial_neighbors(acts_progeny, cutoff=0.1, bandwidth=200, set_diag=False)

## Load metal abundance data

In [ ]:
with open("./grouped_ST_metal_spots_records_edge_filtered.pkl", "rb") as fh:
    metal_abundances = pd.DataFrame.from_records(pickle.load(fh)).drop(columns=["index"])
metal_abundances = sc.read_h5ad("metal_ST.h5ad").obsm["metals"]

## Map metal abundances onto spatial transcriptomics data

In [ ]:
metal_df = pd.DataFrame(
    np.nan, index=adata_st.obsm["mlm_estimate"].index, columns=metal_abundances.columns[1:-2]
)

In [ ]:
metal_df.iloc[metal_abundances["ST_spot_ID"].values, :] = metal_abundances.iloc[:, 1:-2]
adata_st.obsm["metals"] = metal_df
adata_metals = li.ut.obsm_to_adata(adata_st, "metals")

## Preprocess spatial transcriptomics data

In [ ]:
adata_st_new = adata_st.copy()
adata_st_new.var["mt"] = adata_st_new.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata_st_new, qc_vars=["mt"], inplace=True)
sc.pp.filter_genes(adata_st_new, min_cells=0.075 * len(adata_st_new))
adata_st_new.layers['counts'] = adata_st_new.X.copy()
sc.pp.normalize_total(adata_st_new, target_sum=1e4)
sc.pp.log1p(adata_st_new)
sc.pp.highly_variable_genes(adata_st_new, flavor="seurat", n_top_genes=8000)

## Perform spatial autocorrelation analysis

In [ ]:
sq.gr.spatial_autocorr(adata_st_new, seed=42)
sq.gr.spatial_autocorr(adata_st_new, mode="geary", seed=42)

## Define MISTy models

In [ ]:
misty = MistyData(data={
    "cell_types": sc_adata,
    "ST": adata_st_new,
    "intra": adata_metals,
    "Pathways": acts_progeny
})
misty(verbose=True, model=li.method.sp.RandomForestModel, n_jobs=-1, max_depth=6, seed=1337, bypass_intra=True)

## Save results

In [ ]:
misty.write("misty.h5mu")

## Refine results

In [ ]:
final_filtered_keys = ['Ca44', 'Cu63', 'K41', 'Mg24', 'Fe56', 'Zn66', 'Mn55']
for key in ["target_metrics", "interactions"]:
    misty.uns[key] = misty.uns[key][misty.uns[key]["target"].isin(final_filtered_keys)]

In [ ]:
pd.to_pickle({key: misty.uns[key] for key in ["target_metrics", "interactions"]}, "MISTY_results.pkl")

In [ ]:
# Plot interaction metrics
li.pl.target_metrics(misty, stat='gain_R2')